In [1]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [2]:
def extract_frames_from_video(video_path, frame_count=10):
    """
    Extrai um número específico de frames de um vídeo.
    
    :param video_path: Caminho para o vídeo.
    :param frame_count: Número de frames a serem extraídos.
    :return: Lista de frames extraídos.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(total_frames // frame_count, 1)

    for i in range(0, total_frames, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    
    cap.release()
    return frames

In [3]:
def load_video_dataset(data_dir, frame_count=5):
    """
    Carrega o conjunto de dados de vídeos, extrai frames e atribui rótulos.
    
    :param data_dir: Diretório raiz contendo as subpastas dos vídeos.
    :param frame_count: Número de frames a serem extraídos de cada vídeo.
    :return: Arrays de dados e rótulos.
    """
    X, y = [], []
    class_labels = {'YouTube-real': 0, 'Celeb-synthesis': 1, 'Celeb-real': 0}
    
    for label, value in class_labels.items():
        subdir = os.path.join(data_dir, label)
        for file in os.listdir(subdir):
            if file.endswith('.mp4'):  # Supondo que os vídeos são arquivos .mp4
                video_path = os.path.join(subdir, file)
                frames = extract_frames_from_video(video_path, frame_count=frame_count)
                for frame in frames:
                    X.append(frame)
                    y.append(value)
    
    return X, y


In [4]:
def apply_sobel_filter(image, target_size=(256, 256)):
    """
    Aplica o filtro Sobel a uma imagem para detecção de bordas.

    :param image: Imagem a ser processada.
    :param target_size: Tamanho alvo para a imagem processada (largura, altura).
    :return: Imagem com filtro Sobel aplicado.
    """
    try:
        # Converter imagem para escala de cinza
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Aplicar filtros Sobel nas direções x e y
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        
        # Calcular magnitude da borda
        sobel = cv2.magnitude(sobelx, sobely)

        # Redimensionar a imagem para o tamanho alvo
        sobel_resized = cv2.resize(sobel, target_size)

        return sobel_resized
    
    except Exception as e:
        print(f"Erro ao aplicar o filtro Sobel: {e}")
        return None

In [5]:
def process_in_batches(X, batch_size=100, target_size=(256, 256)):
    """
    Processa dados em lotes para reduzir o consumo de memória.

    :param X: Conjunto de dados de imagens.
    :param batch_size: Tamanho do lote.
    :param target_size: Tamanho alvo para as imagens processadas (largura, altura).
    :return: Dados processados.
    """
    X_processed = []
    
    # Dividir dados em lotes e processar cada lote
    for i in range(0, len(X), batch_size):
        batch = X[i:i + batch_size]
        
        # Aplicar filtro Sobel a cada imagem do lote
        batch_sobel = [apply_sobel_filter(frame, target_size) for frame in batch]
        
        # Filtrar imagens processadas corretamente
        batch_sobel = [sobel for sobel in batch_sobel if sobel is not None]
        
        X_processed.extend(batch_sobel)
        
    return np.array(X_processed)


In [6]:
def build_cnn(input_shape):
    """
    Constroi o modelo de CNN.
    
    :param input_shape: Forma da entrada do modelo.
    :return: Modelo CNN compilado.
    """
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [7]:
# Carregar o conjunto de dados de vídeos
data_dir = 'Celeb-DF'
X, y = load_video_dataset(data_dir)

In [8]:
# Processar em lotes
X_sobel = process_in_batches(X, batch_size=100, target_size=(256, 256))

# Normalizar os dados
X_sobel = X_sobel / 255.0

# Expandir as dimensões para incluir o canal de cor
X_sobel = np.expand_dims(X_sobel, axis=-1)

In [9]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_sobel, y, test_size=0.2, random_state=42)

In [10]:
# Definir a forma de entrada
input_shape = (X_sobel.shape[1], X_sobel.shape[2], 1)

# Construir e treinar o modelo CNN
cnn_model = build_cnn(input_shape)
cnn_model.fit(X_train, np.array(y_train), epochs=10, batch_size=32, validation_data=(X_test, np.array(y_test)))

c:\Users\Luca\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 95s 532ms/step - accuracy: 0.7017 - loss: 3.7165 - val_accuracy: 0.8544 - val_loss: 0.4391
Epoch 2/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 90s 520ms/step - accuracy: 0.8616 - loss: 0.3919 - val_accuracy: 0.8717 - val_loss: 0.4107
Epoch 3/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 93s 533ms/step - accuracy: 0.8763 - loss: 0.3468 - val_accuracy: 0.8652 - val_loss: 0.3791
Epoch 4/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 91s 523ms/step - accuracy: 0.8843 - loss: 0.2978 - val_accuracy: 0.8745 - val_loss: 0.3761
Epoch 5/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 91s 524ms/step - accuracy: 0.8971 - loss: 0.2559 - val_accuracy: 0.8868 - val_loss: 0.3562
Epoch 6/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 89s 512ms/step - accuracy: 0.9109 - loss: 0.2237 - val_accuracy: 0.8983 - val_loss: 0.3067
Epoch 7/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 89s 509ms/step - accuracy: 0.9352 - loss: 0.1777 - val_accuracy: 0.9034 - val_loss: 0.2997
Epoch 8/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 87s 500ms/step - accuracy: 0.9398 - loss: 0

In [11]:
cnn_model.save('Model/meu_modelo_sobel.h5')